In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

path = "../Data/FigureS24_ToF/ToF_results.xlsx"
source_dir = "../Data/FigureS24_ToF"

# figure save option
save_option = True
figsavepath1 = "../Figures/FigureS24_ToF_inset.svg"
figsavepath2 = "../Figures/FigureS24_ToF.svg"

df = pd.read_excel(path)
color1 = '#474D6B'

sPV = df['sPV (mm/h)'].to_list()
XRC = df['XRC (arcsec)'].to_list()
mu1 = df['µ'].to_list()
mu2 = df['µ_max'].to_list()
mu3 = df['µ_min'].to_list()
mu_high_error = [m-n for m,n in zip(mu2,mu1)]
mu_low_error = [m-n for m,n in zip(mu1,mu3)]

csv_list = glob.glob(f'{source_dir}/*.csv')
csv_list.sort()

df_list = []
X_list, Y_list = [], []

t_transits = [79.7,52.31,30.96,25.82,21.63,20.93]

colors = ['#CF597E', '#A5698B', '#7C7A98', '#538BA6', '#2A9CB3', '#01ADC1']

### Plot ###
plt.rcParams["font.size"] = 18
fig = plt.figure(figsize=(5,5))

i = 0

for csv in csv_list:
    # get the info
    filename = os.path.basename(csv)
    fileinfo = filename.split('_')
    sample_id = f'{fileinfo[0]}_{fileinfo[1]}' #240220_AY3
    thickness = float(fileinfo[2][:-2]) # mm
    voltage = float(fileinfo[3][:-5]) # V
        
    df = pd.read_csv(csv,header=None,names=['Time(s)','Y'])
    df_list.append(df)
    X = df['Time(s)'].to_list()
    Y = df['Y'].to_list()

    # define t = 0 as Y max
    x_offset = X[Y.index(max(Y))]

    X = [(n-x_offset)*1e6 for n in X] # µs
    Y = [n/max(Y) for n in Y] # normalized
    
    cut_index = Y.index(1)
    
    X = X[cut_index:]
    Y = Y[cut_index:]
    
    X_list.append(X)
    Y_list.append(Y)
    
    i += 1
            
## 1/V time plot ##
d = thickness
voltages = [10,20,40,60,80,100]
inverse_V = [1/n for n in voltages]

plt.scatter(inverse_V,t_transits,c='orange')

# linear regression
coefficients = np.polyfit(inverse_V,t_transits, 1)
slope, intercept = coefficients
X_fit = np.arange(0,0.11,0.01)
Y_fit = [slope*n+intercept for n in X_fit]
plt.plot(X_fit,Y_fit,c='orange',linestyle='dashed')

# prediction
predicted_t_cutoffs = np.polyval(coefficients, inverse_V)

# residual
residuals = np.array(t_transits) - predicted_t_cutoffs

# RSS
RSS = np.sum(residuals**2)

# SXX
SXX = np.sum((np.array(inverse_V) - np.mean(inverse_V))**2)

# Standard error
SE_slope = np.sqrt(RSS / ((len(inverse_V) - 2) * SXX))

mobility = d*d/slope*1e6
mobility_max = (d+0.01)*(d+0.01)/(slope-SE_slope)*1e6
mobility_min = (d-0.01)*(d-0.01)/(slope+SE_slope)*1e6
print(f'\n{sample_id}')
print(f'µ: {mobility:.2f} cm2 V-1 s-1')
print(f'µ min: {mobility_min:.2f} cm2 V-1 s-1')
print(f'µ max: {mobility_max:.2f} cm2 V-1 s-1')

plt.xlabel('1/V (V$^{-1}$)')
plt.ylabel('transit time (µs)')
if save_option:
    plt.savefig(figsavepath1, dpi=1200, bbox_inches='tight', transparent=True)
plt.show()


# Figure plot
plt.rcParams["font.size"] = 18
fig, axes = plt.subplots(nrows=1, ncols=2, tight_layout=True, figsize=(17,7))

ax1 = axes[0]
for i in range(6):
    ax1.plot(X_list[i],Y_list[i],label=f'{int(voltages[i])} V',color=colors[i])
    
    # find transit position
    x_transit = t_transits[i]
    for x, y in zip(X_list[i],Y_list[i]):
        if x > x_transit:
            y_transit = y
            break
    ax1.scatter([x_transit],[y_transit],c='black',s=50,zorder=2)
ax1.set_xlim(1,1000)
ax1.set_ylim(0.1,1)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('Time (µs)')
ax1.set_ylabel('Normalized current (a.u.)')
ax1.legend(frameon=False, loc='upper right',)

ax2 = axes[1]
ax2.errorbar(x=sPV,y=mu1,yerr=[mu_low_error,mu_high_error],linestyle='',marker='o',c=color1)
ax2.set_xlabel('sPV (mm h$^{-1}$)')
ax2.set_ylabel('Mobility (cm$^{2}$ V$^{-1}$ s$^{-1}$)')
ax2.set_xlim(0,0.7)

if save_option:
    plt.savefig(figsavepath2, dpi=1200, bbox_inches='tight', transparent=True)
    
plt.show()